# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хранить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import Counter

In [4]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
news = open('lenta.txt').read()

In [7]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [8]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [9]:
sents_news_train, sents_news_test = train_test_split(sentences_news, test_size=30, random_state=42)

In [10]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sents_news_train:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence, n=3))

In [12]:
matrix_news = lil_matrix((len(bigrams_news), 
                          len(unigrams_news)))


id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

In [13]:
for ngram in trigrams_news:
    bigram, word = ngram.rsplit(' ', 1)
    matrix_news[bigram2id_news[bigram], word2id_news[word]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])
    
matrix_news = csc_matrix(matrix_news)

In [14]:
def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    previous = ['<start>']
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        chosen_w = id2word[chosen]
        text.append(chosen_w)
        
        if chosen_w == '<end>':
            previous = ['<start>']
            chosen_w = '<start>'
            
        previous.append(chosen_w)    
        current_idx = bigram2id[' '.join(previous)]
        previous = previous[1:]
    
    return ' '.join(text)

In [15]:
for i in range(1, 6):
    print(i)
    print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

1
если закон будет принят потребители в случае победы на выборах 
 об этом сегодня сообщил корреспонденту авн что поскольку проверка пока продолжается mars polar lander 
 в принципе считает лужков вариант проведения досрочных президентских выборов кажется достаточно реальной 
 кавказский след считается маловероятным 
 по информации главной военной прокуратурой возбуждено уголовное дело по поводу адекватности правительственных мер 
 крайне сложной и тревожной абсолютно криминальной назвал лидер яблока григорий явлинский 
 коснувшись развернутой в западной европе приближается к миллиарду долларов 
 в его распоряжении есть пленки с записями разговоров которые сделали сами киллеры которые исполняли покушение 
 сейчас у нас будет
2
как отмечается в сообщении кпу повторные президентскиевыборы были проведены широкомасштабные учения “ антитеррор-99 ” для проверки были опрошены все пятеро категорически отказались назвать конкретные города и области могли начаться аварийные автоматические отключ

In [16]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)


def compute_join_proba_markov_assumption(text, bigram_counts, trigram_counts):
    prob = 0
    for trigram in ngrammer(text, 3):
        bigram, word = ngram.rsplit(' ', 1)
        if bigram in bigram_counts and trigram in trigram_counts:
            prob += np.log(trigram_counts[trigram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(text)

In [17]:
ps = []
for sent in sents_news_test:
    prob, N = compute_join_proba_markov_assumption(sent, bigrams_news, trigrams_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [18]:
np.mean(ps)

2841.8666435303926

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [19]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [53]:
def generate_with_beam_search(matrix, id2word, bigram2id, n=100, max_beams=5, start='<start>', prompt = ''):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start] * 2 + normalize(prompt), score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            last_id = bigram2id[' '.join(beam.sequence[-2:])]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            # top_idxs = np.random.choice(matrix.shape[1], 
            #                             size=min(max_beams, probas.astype(bool).sum()),
            #                             p=probas, replace=False)
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    # best_sequence = max(beams, key=lambda x: x.score).sequence
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

In [54]:
generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, max_beams=10)

['<start> <start> как сообщили риа новости <end>',
 '<start> <start> как сообщает риа новости <end>',
 '<start> <start> об этом риа новости <end>',
 '<start> <start> об этом сообщает риа новости <end>',
 '<start> <start> об этом сообщает агентство риа новости <end>',
 '<start> <start> об этом сообщает риа новости со ссылкой на пресс-центр мвд участники совещания заслушали руководителей подразделений на приоритетных направлениях работы защите экономики от криминального влияния ” <end>',
 '<start> <start> об этом сообщает риа новости со ссылкой на пресс-центр объединенной группировки войск на северном кавказе <end>',
 '<start> <start> об этом сообщает риа новости со ссылкой на пресс-центр мвд участники совещания заслушали руководителей подразделений на территории чечни <end>',
 '<start> <start> об этом сообщает риа новости со ссылкой на пресс-центр объединенной группировки федеральных войск на северном кавказе <end>',
 '<start> <start> об этом сообщает риа новости со ссылкой на пресс-цен

In [60]:
generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, max_beams=10, prompt = 'В Москве')

['<start> <start> в москве волгодонске и буйнакске принадлежат к экстремистскому ваххабитскому течению <end>',
 '<start> <start> в москве волгодонске и буйнакске <end>',
 '<start> <start> в москве волгодонске буйнакске и волгодонске <end>',
 '<start> <start> в москве волгодонске и буйнакске принадлежат к местной банде татарин tatarin или братва bratva <end>',
 '<start> <start> в москве и московской области <end>',
 '<start> <start> в москве с 1 января 2000 года <end>',
 '<start> <start> в москве волгодонске и буйнакске принадлежат к сикхской радикальной группировке babbar khalsa занимающейся охотой на ведьм <end>',
 '<start> <start> в москве волгодонске и буйнакске принадлежат к сикхской радикальной группировке babbar khalsa занимающейся охотой на индийских политических лидеров высшего эшелона <end>',
 '<start> <start> в москве по связям с международными финансовыми организациями <end>',
 '<start> <start> в москве волгодонске и буйнакске принадлежат к сикхской радикальной группировке b